In [10]:
import warnings
warnings.filterwarnings("ignore")
from evaluation.mol_structure import list_of_smiles_to_nx_graphs,list_of_nx_graphs_to_smiles
from evaluation.mol_structure import draw_one_mol, draw_graphs
from evaluation.new_structural_metric import symmetric_graph_set_distance,atom,cycle,neighborhood
from evaluate import evaluate
#import utils.graph_generators as gen
import torch
import networkx as nx
import os 
from evaluation.moses.metrics.metrics import get_all_metrics
import sys
import json
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)
import rdkit
import numpy as np
from evaluation.utils import get_data,get_graph_data,  get_mock_data,remove_empty_graphs_and_targets,\
    get_generated_data,preprocess,get_generated_graph_data,_preprocess



# Rewired

In [4]:
import pandas as pd
def get_rewired_graph_data(name,percent):
   try:
     generated_graphs=pd.read_pickle(f'data/rewired/train1/{name}/{name}_rewired_with_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/rewired/train1/{name}/{name}_perturbed_with_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
        target= int(g.nodes[0]['target'])
        targets.append(target)
        for n,i in enumerate(g.nodes(data=True)):
          try:
            g.nodes[n]['label']=int(g.nodes[n]['label'])
          except: None
          try:
           g.nodes[n]['true_label']=int(g.nodes[n]['true_label'])
          except: None
          try:
             g.nodes[n]['attr']=int(g.nodes[n]['true_label'])
          except: None
          del g.nodes[n]['target']

   return  generated_graphs, np.array(targets)

def dict_mean(dict_list):
    mean_dict = {}
    for key in dict_list[0].keys():
      try:
        mean_dict[key] = sum(float(d[key]) for d in dict_list) / len(dict_list)
      except: mean_dict[key]=dict_list[0][key]
    return mean_dict

In [4]:
train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data('synthetic_1_50', path=r'data/graphs/datasets')
a=get_rewired_graph_data('synthetic_1_50',0)
a[0][0].nodes(data=True)

NodeDataView({0: {'label': 1, 'true_label': 1, 'attr': 1}, 1: {'label': 0, 'true_label': 0, 'attr': 0}, 2: {'label': 2, 'true_label': 2, 'attr': 2}, 3: {'label': 0, 'true_label': 0, 'attr': 0}, 4: {'label': 1, 'true_label': 1, 'attr': 1}, 5: {'label': 1, 'true_label': 1, 'attr': 1}, 6: {'label': 0, 'true_label': 0, 'attr': 0}, 7: {'label': 2, 'true_label': 2, 'attr': 2}, 8: {'label': 2, 'true_label': 2, 'attr': 2}, 9: {'label': 1, 'true_label': 1, 'attr': 1}, 10: {'label': 2, 'true_label': 2, 'attr': 2}, 11: {'label': 0, 'true_label': 0, 'attr': 0}, 12: {'label': 2, 'true_label': 2, 'attr': 2}, 13: {'label': 2, 'true_label': 2, 'attr': 2}, 14: {'label': 1, 'true_label': 1, 'attr': 1}, 15: {'label': 2, 'true_label': 2, 'attr': 2}, 16: {'label': 0, 'true_label': 0, 'attr': 0}, 17: {'label': 0, 'true_label': 0, 'attr': 0}, 18: {'label': 0, 'true_label': 0, 'attr': 0}, 19: {'label': 0, 'true_label': 0, 'attr': 0}, 20: {'label': 1, 'true_label': 1, 'attr': 1}, 21: {'label': 1, 'true_label':

In [3]:


structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=['0','5','10','25','33','40','50','66','75','90','100']
metrics={}
for generator in ['rewired']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            for percent in rewiring_ratio:
                a=[]
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_rewired_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_1_50
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Now computing the auc_roc based  metric
0.73925 0.8126 0.7620000000000001 0.8126
{'activations_time': 0.039585018157958986, 'fid': 10.097126382598908, 'fid_time': 0.04147427876790365, 'kid': 91.85071109135946, 'kid_time': 0.3353947321573893, 'precision': 0.9643333333333334, 'recall': 0.9778333333333332, 'f1_pr': 0.9709917770005424, 'precision_time': 0.09449907938639322, 'recall_time': 0.09449907938639322, 'f1_pr_time': 0.09449907938639322, 'density': 1.0061333333333333, 'coverage': 0.9761666666666666, 'f1_dc': 0.9899602898144073, 'density_time': 0.07330597241719564, 'coverage_time': 0.07330597241719564, 'f1_dc_time': 0.07330597241719564, 'mmd_rbf': 0.0052900259072581925, 'mmd_rbf_time': 0.105745538075

# Pertubed with ER

In [11]:

import pandas as pd
def get_corrupted_graph_data(name,percent):
   try:
     generated_graphs=pd.read_pickle(f'data/corrupted/train1/{name}/{name}_corrupted_with_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/corrupted/train1/{name}/{name}_perturbed_with_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
        target= int(g.nodes[0]['target'])
        targets.append(target)
        for n,i in enumerate(g.nodes(data=True)):
          try:
            g.nodes[n]['label']=int(g.nodes[n]['label'])
          except: None
          try:
           g.nodes[n]['true_label']=int(g.nodes[n]['true_label'])
          except: None
          try:
             g.nodes[n]['attr']=[int(l) for l in list(g.nodes[n]['attr'])]
          except: None
          del g.nodes[n]['target']
   return  generated_graphs, np.array(targets)


def dict_mean(dict_list):
    mean_dict = {}
    for key in dict_list[0].keys():
      try:
        mean_dict[key] = sum(float(d[key]) for d in dict_list) / len(dict_list)
      except: mean_dict[key]=dict_list[0][key]
    return mean_dict

In [29]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['corrupted']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                try:
                    a=[]
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_corrupted_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                except:continue

ames_50 corrupted
Now computing classifier based metrics
fid calculation produces singular product; adding 1e-06 to diagonal of cov estimates
Now computing structural based metrics
{'activations_time': 0.24522412618001302, 'fid': 70.38451679757277, 'fid_time': 0.2489896297454834, 'kid': -6.351243938816091, 'kid_time': 0.3549291292826335, 'precision': 0.9697945295118439, 'recall': 0.973406622169873, 'f1_pr': 0.9716025799855341, 'precision_time': 0.6804068565368653, 'recall_time': 0.6804068565368653, 'f1_pr_time': 0.6804068565368653, 'density': 1.0018322209134929, 'coverage': 0.9684072765344851, 'f1_dc': 0.9848244866673347, 'density_time': 0.5052577177683513, 'coverage_time': 0.5052577177683513, 'f1_dc_time': 0.5052577177683513, 'mmd_rbf': 0.000810229766163199, 'mmd_rbf_time': 1.1824428717295328, 'mmd_linear': 0.09989025046428045, 'mmd_linear_time': 0.2453673521677653, 'wl_mmd': 0.0007852375343541421, 'wl_mmd_time': 2.7619361877441406, 'nspdk_mmd': 0.0006358862277980881, 'nspdk_mmd_time'

In [14]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['corrupted']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                
                    a=[]
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_corrupted_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)


ames_50 corrupted
Now computing structural based metrics
{'wl_mmd': 0.00040676095038549853, 'wl_mmd_time': 2.2033088207244873, 'dataset': 'ames_50', 'generator_name': 'corrupted', 'degree_of_perturbation': 0}
ames_50 corrupted
Now computing structural based metrics
{'wl_mmd': 0.0008789668841170384, 'wl_mmd_time': 2.0561721324920654, 'dataset': 'ames_50', 'generator_name': 'corrupted', 'degree_of_perturbation': 0.05}
ames_50 corrupted
Now computing structural based metrics
{'wl_mmd': 0.0023488934084208157, 'wl_mmd_time': 2.090056896209717, 'dataset': 'ames_50', 'generator_name': 'corrupted', 'degree_of_perturbation': 0.1}
ames_50 corrupted
Now computing structural based metrics
{'wl_mmd': 0.004436021418562142, 'wl_mmd_time': 2.0459980964660645, 'dataset': 'ames_50', 'generator_name': 'corrupted', 'degree_of_perturbation': 0.15}
ames_50 corrupted
Now computing structural based metrics
{'wl_mmd': 0.01261978494493976, 'wl_mmd_time': 2.056595802307129, 'dataset': 'ames_50', 'generator_name'

In [4]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural','auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['corrupted']:
    for dataset_name in ['synthetic_2','synthetic_1']:
        for split in splits:
            for percent in rewiring_ratio:
                
                    a=[]
                    print(dataset_name+split, percent)
                    train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                    generated_graphs, generated_targets= get_corrupted_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)


synthetic_2_50 0
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Now computing the auc_roc based  metric
0.82495 0.82805 0.8065000000000001 0.82805
{'activations_time': 0.0387423038482666, 'fid': 0.42309173315282844, 'fid_time': 0.04032015800476074, 'kid': 0.006520263353983561, 'kid_time': 0.339206059773763, 'precision': 0.9776666666666666, 'recall': 0.9808333333333333, 'f1_pr': 0.979224913839528, 'precision_time': 0.09901529947916667, 'recall_time': 0.09901529947916667, 'f1_pr_time': 0.09901529947916667, 'density': 0.9774999999999998, 'coverage': 0.9488333333333332, 'f1_dc': 0.9617888190885137, 'density_time': 0.07264540990193685, 'coverage_time': 0.07264540990193685, 'f1_dc_time': 0.07264540990193685, 'mmd_rbf': 0.005727538466453552, 'mmd_rbf_time': 0.105483643

In [ ]:
0.73925 0.8126 0.7620000000000001 0.8126


In [ ]:
0.7620000000000001 0.8126 0.71085 0.7885000000000001


In [7]:
train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data('synthetic_1_50', path=r'data/graphs/datasets')
generated_graphs, generated_targets= get_corrupted_graph_data('synthetic_1_50',int(0*100))

In [9]:
train1_graphs[0].nodes(data=True)

NodeDataView({0: {'true_label': 1, 'label': 0}, 1: {'true_label': 0, 'label': 0}, 2: {'true_label': 2, 'label': 0}, 3: {'true_label': 0, 'label': 0}, 4: {'true_label': 1, 'label': 0}, 5: {'true_label': 1, 'label': 0}, 6: {'true_label': 0, 'label': 0}, 7: {'true_label': 2, 'label': 0}, 8: {'true_label': 2, 'label': 0}, 9: {'true_label': 1, 'label': 0}, 10: {'true_label': 2, 'label': 0}, 11: {'true_label': 0, 'label': 0}, 12: {'true_label': 2, 'label': 0}, 13: {'true_label': 2, 'label': 0}, 14: {'true_label': 1, 'label': 0}, 15: {'true_label': 2, 'label': 0}, 16: {'true_label': 0, 'label': 0}, 17: {'true_label': 0, 'label': 0}, 18: {'true_label': 0, 'label': 0}, 19: {'true_label': 0, 'label': 0}, 20: {'true_label': 1, 'label': 0}, 21: {'true_label': 0, 'label': 0}, 22: {'true_label': 1, 'label': 0}, 23: {'true_label': 0, 'label': 0}, 24: {'true_label': 2, 'label': 0}, 25: {'true_label': 2, 'label': 0}, 26: {'true_label': 0, 'label': 0}, 27: {'true_label': 2, 'label': 0}, 28: {'true_label

In [8]:
generated_graphs[0].nodes(data=True)

NodeDataView({0: {'label': 0}, 1: {'label': 0}, 2: {'label': 0}, 3: {'label': 0}, 4: {'label': 0}, 5: {'label': 0}, 6: {'label': 0}, 7: {'label': 0}, 8: {'label': 0}, 9: {'label': 0}, 10: {'label': 0}, 11: {'label': 0}, 12: {'label': 0}, 13: {'label': 0}, 14: {'label': 0}, 15: {'label': 0}, 16: {'label': 0}, 17: {'label': 0}, 18: {'label': 0}, 19: {'label': 0}, 20: {'label': 0}, 21: {'label': 0}, 22: {'label': 0}, 23: {'label': 0}, 24: {'label': 0}, 25: {'label': 0}, 26: {'label': 0}, 27: {'label': 0}, 28: {'label': 0}, 29: {'label': 0}})

# Mode Dropping


In [ ]:
import pandas as pd
def get_mode_dropping_graph_data(name,percent,return_smiles=False):
   try:
     generated_graphs=pd.read_pickle(f'data/mode_dropping/train1/{name}/{name}_no_of_clusters_dropped_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/mode_dropping/train1/{name}/{name}_fraction_of_clusters_dropped_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
      try:
        target= int(g.nodes[0]['target'])
        targets.append(target)
      except:None
      
      for n,i in enumerate(g.nodes(data=True)):
        try:
          g.nodes[n]['label']=int(g.nodes[n]['label'])
        except: None
        try:
          g.nodes[n]['true_label']=int(g.nodes[n]['true_label'])
        except: None
        try:
            g.nodes[n]['attr']=[int(l) for l in list(g.nodes[n]['attr'])]
        except: None
        try:  del g.nodes[n]['target']
        except:None
   if return_smiles:
      return(list_of_nx_graphs_to_smiles(generated_graphs))
   return  generated_graphs, np.array(targets)

def dict_mean(dict_list):
    mean_dict = {}
    for key in dict_list[0].keys():
      try:
        mean_dict[key] = sum(float(d[key]) for d in dict_list) / len(dict_list)
      except: mean_dict[key]=dict_list[0][key]
    return mean_dict

In [10]:
generated=get_mode_dropping_graph_data('synthetic_1_50',0)

## Synthetic_1

In [6]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural','auc_roc']
generators=['swingnn','gdss']
splits=['_50']
r=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
rewiring_ratio=r
metrics={}
for generator in ['mode_dropping']:
    for dataset_name in ['synthetic_1']:
        for split in splits:
            for percent in r:
                a=[]
                print(dataset_name+split, percent)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_dropping_graph_data(dataset_name+split,int(percent*100))
                if len(generated_targets)==0:
                    generated_targets=train1_targets
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)



synthetic_1_50 0
Now computing structural based metrics
Now computing the auc_roc based  metric
0.73925 0.8126 0.7620000000000001 0.8126
{'activations_time': 0.04059147834777832, 'fid': 0.06130852750950666, 'fid_time': 0.041969919204711915, 'kid': -0.003756586710611979, 'kid_time': 0.05698911348978678, 'precision': 0.9676666666666667, 'recall': 0.9811666666666666, 'f1_pr': 0.9743403684074234, 'precision_time': 0.14543458620707195, 'recall_time': 0.14543458620707195, 'f1_pr_time': 0.14543458620707195, 'density': 0.9930333333333333, 'coverage': 0.9701666666666665, 'f1_dc': 0.9812032934691429, 'density_time': 0.09930488268534342, 'coverage_time': 0.09930488268534342, 'f1_dc_time': 0.09930488268534342, 'mmd_rbf': 0.005474048643372953, 'mmd_rbf_time': 0.13146387736002604, 'mmd_linear': 0.04429222792387009, 'mmd_linear_time': 0.04063458442687988, 'wl_mmd': 0.0024584148809490003, 'wl_mmd_time': 0.4435436725616455, 'nspdk_mmd': 0.004186284402300344, 'nspdk_mmd_time': 9.034395933151245, 'degree

## Synthetic_2

In [7]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural','auc_roc']
generators=['swingnn','gdss']
splits=['_50']
r=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
rewiring_ratio=r
metrics={}
for generator in ['mode_dropping']:
    for dataset_name in ['synthetic_2']:
        for split in splits:
            for percent in r:
                a=[]
                print(dataset_name+split, percent)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_dropping_graph_data(dataset_name+split,int(percent*100))
                if len(generated_targets)==0:
                    generated_targets=train1_targets
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)



synthetic_2_50 0
Now computing structural based metrics
Now computing the auc_roc based  metric
0.82495 0.82805 0.8065000000000001 0.82805
{'activations_time': 0.039417266845703125, 'fid': 0.35190771839634605, 'fid_time': 0.04444378217061361, 'kid': 0.005726750691731771, 'kid_time': 0.06160295804341634, 'precision': 0.9695, 'recall': 0.9834999999999999, 'f1_pr': 0.9764192463967077, 'precision_time': 0.15286744435628255, 'recall_time': 0.15286744435628255, 'f1_pr_time': 0.15286744435628255, 'density': 0.9220333333333333, 'coverage': 0.9373333333333332, 'f1_dc': 0.9294056012675455, 'density_time': 0.10478692054748535, 'coverage_time': 0.10478692054748535, 'f1_dc_time': 0.10478692054748535, 'mmd_rbf': 0.005667181468258301, 'mmd_rbf_time': 0.1434610366821289, 'mmd_linear': 0.13392266482114792, 'mmd_linear_time': 0.039458115895589195, 'wl_mmd': 0.0016914465204012075, 'wl_mmd_time': 0.45431089401245117, 'nspdk_mmd': 0.003687769631975757, 'nspdk_mmd_time': 8.888182163238525, 'degree_mmd': 6.6

## Smiles Datasets


###  NN and structural

In [31]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural', 'auc_roc']
#leave it blank to compute all the structural structures 
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['mode_dropping']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                try:
                    a=[]
                    print(dataset_name+split, percent)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_mode_dropping_graph_data(dataset_name+split,int(percent*100))
                    if len(generated_targets)==0:
                        generated_targets=train1_targets
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)  
                    metrics=dict_mean(a)        
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                except:continue

ames_50 0
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.8829444431748061 0.9113656100738168 0.8856954315880887 0.9113656100738168
{'activations_time': 0.24741024971008302, 'fid': 773.5298233629668, 'fid_time': 0.2511648178100586, 'kid': 231314.46534095853, 'kid_time': 0.36671841939290367, 'precision': 0.973956288443921, 'recall': 0.9750032718230598, 'f1_pr': 0.9744846031414801, 'precision_time': 0.6604457855224609, 'recall_time': 0.6604457855224609, 'f1_pr_time': 0.6604457855224609, 'density': 0.9998377175762335, 'coverage': 0.9701609736945426, 'f1_dc': 0.9847609137331064, 'density_time': 0.49517410596211753, 'coverage_time': 0.49517410596211753, 'f1_dc_time': 0.49517410596211753, 'mmd_rbf': 0.0008284938938838119, 'mmd_rbf_time': 1.1660237312316895, 'mmd_linear': 0.6571949948867162, 'mmd_linear_time': 0.2475509484608968, 'wl_mmd': 0.0007852375343541421, 'wl_mmd_time': 3.0865519046783447, 'nspdk_mmd': 0.0006358862

In [17]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['mode_dropping']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
            for percent in rewiring_ratio:
                try:
                    a=[]
                    print(dataset_name+split, percent)
                    generated_graphs, generated_targets=get_mode_dropping_graph_data(dataset_name+split,int(percent*100))
                    if len(generated_targets)==0:
                        generated_targets=train1_targets
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]        
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                except:continue

ames_50 0
Now computing structural based metrics
{'wl_mmd': 0.00040676095038505444, 'wl_mmd_time': 2.516537666320801, 'dataset': 'ames_50', 'generator_name': 'mode_dropping', 'degree_of_perturbation': 0}
ames_50 0.05
Now computing structural based metrics
{'wl_mmd': 0.0005047118488226143, 'wl_mmd_time': 2.4105846881866455, 'dataset': 'ames_50', 'generator_name': 'mode_dropping', 'degree_of_perturbation': 0.05}
ames_50 0.1
Now computing structural based metrics
{'wl_mmd': 0.0004461698968944994, 'wl_mmd_time': 2.543876886367798, 'dataset': 'ames_50', 'generator_name': 'mode_dropping', 'degree_of_perturbation': 0.1}
ames_50 0.15
Now computing structural based metrics
{'wl_mmd': 0.0004364185894121242, 'wl_mmd_time': 2.541646718978882, 'dataset': 'ames_50', 'generator_name': 'mode_dropping', 'degree_of_perturbation': 0.15}
ames_50 0.25
Now computing structural based metrics
{'wl_mmd': 0.0006605779647423926, 'wl_mmd_time': 2.51676082611084, 'dataset': 'ames_50', 'generator_name': 'mode_dropp

### Molecular metrics

In [ ]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['corrupted']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                
                    print(dataset_name+split, generator)
                    graphs, splits  =get_data(dataset_name+split, return_smiles=True)
                    generated_smiles=get_mode_dropping_graph_data(dataset_name+split,int(percent*100),return_smiles=True)
                    train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
                    metrics=get_all_metrics(gen=generated_smiles,train=train2_smiles,test=train2_smiles)
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)

# Mode Collapse

In [12]:




import pandas as pd
def get_mode_collapse_graph_data(name,percent,return_smiles=False):
   try:
     generated_graphs=pd.read_pickle(f'data/mode_collapse/train1/{name}/{name}_no_of_clusters_collapsed_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/mode_collapse/train1/{name}/{name}_fraction_of_clusters_collapsed_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
      try:
        target= int(g.nodes[0]['target'])
        targets.append(target)
      except:None
      
      for n,i in enumerate(g.nodes(data=True)):
        try:
          g.nodes[n]['label']=int(g.nodes[n]['label'])
        except: None
        try:
          g.nodes[n]['true_label']=int(g.nodes[n]['true_label'])
        except: None
        try:
            g.nodes[n]['attr']=[int(l) for l in list(g.nodes[n]['attr'])]
        except: None
        try:  del g.nodes[n]['target']
        except:None
   if return_smiles:
      return(list_of_nx_graphs_to_smiles(generated_graphs)) 
   return  generated_graphs, np.array(targets)

## Sythetic_1

In [9]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural','auc_roc']
generators=['swingnn','gdss']
splits=['_50']
r=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
rewiring_ratio=r
metrics={}
for generator in ['mode_collapse']:
    for dataset_name in ['synthetic_1']:
        for split in splits:
            for percent in r:
                a=[]
                print(dataset_name+split, percent)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_collapse_graph_data(dataset_name+split,int(percent*100))
                if len(generated_targets)==0:
                    generated_targets=train1_targets
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)


synthetic_1_50 0
Now computing structural based metrics
Now computing the auc_roc based  metric
0.73925 0.8126 0.7620000000000001 0.8126
{'activations_time': 0.04258019129435221, 'fid': 0.09402447297071319, 'fid_time': 0.04406140645345052, 'kid': -0.0039353609085083004, 'kid_time': 0.058643738428751625, 'precision': 0.9725000000000004, 'recall': 0.9818333333333332, 'f1_pr': 0.9771386189224093, 'precision_time': 0.09841585159301758, 'recall_time': 0.09841585159301758, 'f1_pr_time': 0.09841585159301758, 'density': 0.9862666666666667, 'coverage': 0.9764999999999998, 'f1_dc': 0.9812069984287801, 'density_time': 0.09647760391235352, 'coverage_time': 0.09647760391235352, 'f1_dc_time': 0.09647760391235352, 'mmd_rbf': 0.005234299856238067, 'mmd_rbf_time': 0.12937671343485516, 'mmd_linear': 0.04704691171646118, 'mmd_linear_time': 0.04263126055399577, 'wl_mmd': 0.0024584148809490003, 'wl_mmd_time': 0.512505054473877, 'nspdk_mmd': 0.004186284402300344, 'nspdk_mmd_time': 9.036637306213379, 'degree

## Synthetic_2

In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural','auc_roc']
generators=['swingnn','gdss']
splits=['_50']
r=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
rewiring_ratio=r
metrics={}
for generator in ['mode_collapse']:
    for dataset_name in ['synthetic_2']:
        for split in splits:
            for percent in r:
                a=[]
                print(dataset_name+split, percent)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_collapse_graph_data(dataset_name+split,int(percent*100))
                if len(generated_targets)==0:
                    generated_targets=train1_targets
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)


synthetic_2_50 0
Now computing structural based metrics
Now computing the auc_roc based  metric
0.82495 0.82805 0.8065000000000001 0.82805
{'activations_time': 0.03945895830790202, 'fid': 0.31408806280814333, 'fid_time': 0.04107472101847331, 'kid': 0.0062688668568929035, 'kid_time': 0.0557042121887207, 'precision': 0.9771666666666666, 'recall': 0.9819999999999999, 'f1_pr': 0.9795468115425034, 'precision_time': 0.1433971087137858, 'recall_time': 0.1433971087137858, 'f1_pr_time': 0.1433971087137858, 'density': 0.9593000000000002, 'coverage': 0.9516666666666665, 'f1_dc': 0.9549409343229001, 'density_time': 0.09858605066935221, 'coverage_time': 0.09858605066935221, 'f1_dc_time': 0.09858605066935221, 'mmd_rbf': 0.005827442665273945, 'mmd_rbf_time': 0.12752685546875, 'mmd_linear': 0.1416512221097946, 'mmd_linear_time': 0.039501221974690755, 'wl_mmd': 0.0016914465204012075, 'wl_mmd_time': 0.48560404777526855, 'nspdk_mmd': 0.003687769631975757, 'nspdk_mmd_time': 8.893813848495483, 'degree_mmd'

## Smiles Datasets

### NN and structural

In [8]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['mode_collapse']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                try:
                    a=[]
                    print(dataset_name+split, percent)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_mode_collapse_graph_data(dataset_name+split,int(percent*100))
                    if len(generated_targets)==0:
                        generated_targets=train1_targets
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)  
                    metrics=dict_mean(a)        
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                except:continue

ames_50 0
Now computing classifier based metrics
Now computing structural based metrics
{'activations_time': 0.23974318504333497, 'fid': 1316.1211831729831, 'fid_time': 0.2656317075093587, 'kid': -5123.208109294095, 'kid_time': 0.34620687166849773, 'precision': 0.9704750687082841, 'recall': 0.9714173537495091, 'f1_pr': 0.9709497293690031, 'precision_time': 0.9422141234079997, 'recall_time': 0.9422141234079997, 'f1_pr_time': 0.9422141234079997, 'density': 0.9931317890328493, 'coverage': 0.9679884831828294, 'f1_dc': 0.980380534285233, 'density_time': 0.7025370121002197, 'coverage_time': 0.7025370121002197, 'f1_dc_time': 0.7025370121002197, 'mmd_rbf': 0.0008052544003779379, 'mmd_rbf_time': 1.5013645966847737, 'mmd_linear': 0.9003139207760493, 'mmd_linear_time': 0.2398860772450765, 'wl_mmd': 0.00040676095038505444, 'wl_mmd_time': 2.442007064819336, 'nspdk_mmd': 0.0006330380990651197, 'nspdk_mmd_time': 15.055759906768799, 'degree_mmd': 3.204833337488999e-05, 'degree_mmd_time': 38.522282, 'c

In [9]:
current_smiles_datasets=['cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['mode_collapse']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                try:
                    a=[]
                    print(dataset_name+split, percent)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_mode_collapse_graph_data(dataset_name+split,int(percent*100))
                    if len(generated_targets)==0:
                        generated_targets=train1_targets
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)  
                    metrics=dict_mean(a)        
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                except:continue

cyp2c19_veith_50 0
Now computing classifier based metrics
Now computing structural based metrics
{'activations_time': 0.4122882843017578, 'fid': 6.896770001759152, 'fid_time': 0.42931405703226727, 'kid': -3.4672299042344092, 'kid_time': 0.5383693059285481, 'precision': 0.9638769832318219, 'recall': 0.9637334536702767, 'f1_pr': 0.9638108138164024, 'precision_time': 1.930309788386027, 'recall_time': 1.930309788386027, 'f1_pr_time': 1.930309788386027, 'density': 0.9909827806602001, 'coverage': 0.9695397111913359, 'f1_dc': 0.9801373318851304, 'density_time': 1.304549789428711, 'coverage_time': 1.304549789428711, 'f1_dc_time': 1.304549789428711, 'mmd_rbf': 0.0005243629217147827, 'mmd_rbf_time': 3.3429211457570394, 'mmd_linear': 0.05617493242025375, 'mmd_linear_time': 0.41249629656473796, 'wl_mmd': 0.0001614312225315384, 'wl_mmd_time': 8.081013917922974, 'nspdk_mmd': 0.0003402020997011479, 'nspdk_mmd_time': 42.42992305755615, 'degree_mmd': 8.808374595048463e-06, 'degree_mmd_time': 121.937307

### Molecular metrics

In [ ]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['corrupted']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                
                    print(dataset_name+split, generator)
                    graphs, splits  =get_data(dataset_name+split, return_smiles=True)
                    generated_smiles=get_mode_collapse_graph_data(dataset_name+split,int(percent*100),return_smiles=True)
                    train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
                    metrics=get_all_metrics(gen=generated_smiles,train=train2_smiles,test=train2_smiles)
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)

In [16]:
g=get_mode_collapse_graph_data(dataset_name+split,int(percent*100),return_smiles=False)

In [24]:
g[0][0].edges(data=True)

EdgeDataView([(0, 1, {'id': 0, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (0, 13, {'id': 1, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (1, 2, {'id': 3, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (2, 3, {'id': 5, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (2, 14, {'id': 6, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (3, 4, {'id': 8, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (4, 5, {'id': 10, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (5, 6, {'id': 12, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (5, 15, {'id': 13, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (6, 7, {'id': 15, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (7, 8, {'id': 17, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (8, 9, {'id': 19, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (9, 10, {'id': 21, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (9, 15, {'id': 22, 'label': tensor(1), 'attr': tensor([3, 0, 1])}), (10, 11, {'id': 24, 'label': tensor(1), 'attr': t

## Additionally train2 versus train2

In [7]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural', 'auc_roc']
#leave it blank to compute all the structural structures 
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['train2']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
                
                    a=[]
                    print(dataset_name+split)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=train2_graphs , train2_targets
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)  
                    metrics=dict_mean(a)        
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    print(metrics)
              

ames_50
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Now computing the auc_roc based  metric
0.8829444431748061 0.9113656100738168 0.8829444431748061 0.8829444431748061
{'activations_time': 0.2121817111968994, 'fid': -4.575788163189524e-09, 'fid_time': 0.21556973457336426, 'kid': -0.11167725622653961, 'kid_time': 0.6435142676035563, 'precision': 1.0, 'recall': 1.0, 'f1_pr': 1.0000099999999998, 'precision_time': 0.6262365341186523, 'recall_time': 0.6262365341186523, 'f1_pr_time': 0.6262365341186523, 'density': 1.2005130218557782, 'coverage': 1.0, 'f1_dc': 1.091131107316819, 'density_time': 0.45873883565266926, 'coverage_time': 0.45873883565266926, 'f1_dc_time': 0.45873883565266926, 'mmd_rbf': 0.0, 'mmd_rbf_time': 1.1450655

# SWINGNN as we increase the number of training epochs

In [17]:

import pandas as pd
from glob import glob
from sklearn.utils import shuffle                                                                                                                                                                                                                                                                                                                                                                                                                                                              
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.utils import remove_empty_graphs_and_targets
import numpy as np

def dict_mean(dict_list):
    mean_dict = {}
    for key in dict_list[0].keys():
      try:
        mean_dict[key] = sum(float(d[key]) for d in dict_list) / len(dict_list)
      except: mean_dict[key]=dict_list[0][key]
    return mean_dict 
def get_swingnn_graph_data(name,percent,split,path='data/SWINGNN_experiments/'):
     path+=f'{percent}-percent/'
     path_postives=path+r'{}/{}/pos/*.pkl'.format(name,name+split)
     files = glob(path_postives)
     generated_pos_graphs=pd.read_pickle(files[0])
     path_negatives=path+r'{}/{}/neg/*.pkl'.format(name,name+split)
     files = glob(path_negatives)
     generated_neg_graphs=pd.read_pickle(files[0])
     generated_graphs,generated_targets = generated_pos_graphs + generated_neg_graphs, [1]*len(generated_pos_graphs)+[0]*len(generated_neg_graphs)

     return  generated_graphs, np.array(generated_targets)

In [18]:
from glob import glob
from sklearn.utils import shuffle                                                                                                                                                                                                                                                                                                                                                                                                                                                              
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.utils import remove_empty_graphs_and_targets
import numpy as np

def get_swingnn_molecular_data(name,percent, path='data/SWINGNN_experiments/', generator_name='stgg',return_smiles=False):
     path+=f'{percent}-percent/'
     pos_list, neg_list=[],[]
     path_postives=path+r'{}/{}/pos/*.txt'.format(name,name+'_50')
     files = glob(path_postives)
     
     with open(files[0]) as my_file:
          for line in my_file:
               pos_list.append(line.strip())

     path_negatives=path+r'{}/{}/neg/*.txt'.format(name,name+'_50')
     files = glob(path_negatives)
     
     with open(files[0]) as my_file:
          for line in my_file:
               neg_list.append(line.strip())

     pos_graphs=list_of_smiles_to_nx_graphs(pos_list)
     neg_graphs=list_of_smiles_to_nx_graphs(neg_list)
     #print(len(neg_graphs))
     generated_graphs,generated_targets = pos_graphs + neg_graphs, [1]*len(pos_graphs)+[0]*len(neg_graphs)
     #generated_graphs, generated_targets = shuffle(generated_graphs, generated_targets)
     generated_graphs, generated_targets = remove_empty_graphs_and_targets(generated_graphs, generated_targets)
     if return_smiles:
          return  generated_graphs, generated_targets,pos_list+neg_list
     return  generated_graphs, np.array(generated_targets)

## Molecular graphs

### nn and structural 

In [23]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_25','_33','_40','_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['swingnn']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
split='_50'
for generator in generators:
    for dataset_name in current_smiles_datasets:
             for percent in experiments:
                a=[]
                print(dataset_name+split)
                train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                generated_graphs, generated_targets=get_swingnn_molecular_data(dataset_name,percent)
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                print('Now computing classifier based metrics')
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)  
                metrics=dict_mean(a)        
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100
                print(metrics)

ames_50
Now computing classifier based metrics
fid calculation produces singular product; adding 1e-06 to diagonal of cov estimates
fid calculation produces singular product; adding 1e-06 to diagonal of cov estimates
Now computing structural based metrics
{'activations_time': 0.22230366071065266, 'fid': 44.84608418311873, 'fid_time': 0.22768174807230632, 'kid': 18.533125587304433, 'kid_time': 0.32477649052937824, 'precision': 0.8592854338437377, 'recall': 0.607826200759063, 'f1_pr': 0.7108496695562795, 'precision_time': 0.6160453955332438, 'recall_time': 0.6160453955332438, 'f1_pr_time': 0.6160453955332438, 'density': 0.6360371679099595, 'coverage': 0.3174977097238581, 'f1_dc': 0.4225460513940954, 'density_time': 0.4626341978708903, 'coverage_time': 0.4626341978708903, 'f1_dc_time': 0.4626341978708903, 'mmd_rbf': 0.17494341532389324, 'mmd_rbf_time': 1.1218075275421142, 'mmd_linear': 17.63351993560791, 'mmd_linear_time': 0.222443151473999, 'wl_mmd': 0.12603377436594054, 'wl_mmd_time': 2

### train2 vs train1

In [24]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train1']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
split='_50'
for generator in generators:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            a=[]
            print(dataset_name+split)
            train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
            generated_graphs, generated_targets=train1_graphs , train1_targets
            splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
            print('Now computing classifier based metrics')
            for run in range(15):
                metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                a.append(metrics)  
            metrics=dict_mean(a)        
            metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_25
Now computing classifier based metrics
Now computing structural based metrics
{'activations_time': 0.10143287976582845, 'fid': 676.4820753139189, 'fid_time': 0.1031204064687093, 'kid': 23690.17833514611, 'kid_time': 0.1587807019551595, 'precision': 0.9717727153705159, 'recall': 0.964545692589683, 'f1_pr': 0.9681448535445277, 'precision_time': 0.2486522356669108, 'recall_time': 0.2486522356669108, 'f1_pr_time': 0.2486522356669108, 'density': 1.0045456925896834, 'coverage': 0.9634459282534693, 'f1_dc': 0.9835358342869652, 'density_time': 0.22519830067952473, 'coverage_time': 0.22519830067952473, 'f1_dc_time': 0.22519830067952473, 'mmd_rbf': 0.00204785813887914, 'mmd_rbf_time': 0.40147164662679036, 'mmd_linear': 0.7088802725076675, 'mmd_linear_time': 0.1015286922454834, 'wl_mmd': 0.0011618091964094024, 'wl_mmd_time': 0.9091582298278809, 'nspdk_mmd': 0.0014439711207633876, 'nspdk_mmd_time': 7.234716892242432, 'degree_mmd': 0.0002635944489091102, 'degree_mmd_time': 14.757093, 'clust

### molecular

In [19]:
from evaluation.moses.metrics.metrics import get_all_metrics
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split1='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['swingnn']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            for percent in experiments:
                print(dataset_name+split1)
                graphs, splits =get_data(dataset_name+split1, return_smiles=True)
                train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
                _,generated_targets, generated_smiles=get_swingnn_molecular_data(dataset_name,percent,return_smiles=True) 
                metrics=get_all_metrics(gen=generated_smiles,train=train2_smiles,test=train2_smiles)
                metrics['dataset']=dataset_name+split1
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100 
                print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.992, 'unique@10000': 0.9752650176678446, 'FCD/Test': 17.196146608702655, 'SNN/Test': 0.27051011988074813, 'Frag/Test': 0.1531511401333041, 'Scaf/Test': 0.05274522872358611, 'FCD/TestSF': 37.4445566284182, 'SNN/TestSF': 0.19734603975408443, 'Frag/TestSF': 0.10150397926772925, 'Scaf/TestSF': 4.1041894040527716e-05, 'IntDiv': 0.9190337370237776, 'IntDiv2': 0.9022144090456522, 'Filters': 0.5127601099332548, 'logP': 0.19126302709069487, 'SA': 1.4890440133925626, 'QED': 0.08059893151943093, 'weight': 54.34200942285044, 'Novelty': 0.9967793880837359, 'dataset': 'ames_50', 'generator_name': 'swingnn', 'numer_of_epochs_trained': 250.0}
ames_50
{'valid': 1.0, 'unique@1000': 0.987, 'unique@10000': 0.9733019238319591, 'FCD/Test': 15.621305317824671, 'SNN/Test': 0.28040955265129036, 'Frag/Test': 0.19670744762450332, 'Scaf/Test': 0.07717364804524973, 'FCD/TestSF': 35.64221421636759, 'SNN/TestSF': 0.21063697174583457, 'Frag/TestSF': 0.14979314122713538, 'Scaf/T

#### train1


In [20]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split ='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train1']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            print(dataset_name+split)
            graphs, splits =get_data(dataset_name+split, return_smiles=True)
            train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
            train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
            metrics=get_all_metrics(gen=train1_smiles,train=train2_smiles,test=train2_smiles)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.999, 'unique@10000': 0.9996073812328229, 'FCD/Test': 0.8156919778498235, 'SNN/Test': 0.575357792673038, 'Frag/Test': 0.9878820451873492, 'Scaf/Test': 0.8593618010840082, 'FCD/TestSF': 22.270569671809774, 'SNN/TestSF': 0.33544746274439363, 'Frag/TestSF': 0.7510482960545906, 'Scaf/TestSF': 0.13905682249091533, 'IntDiv': 0.9105206593714188, 'IntDiv2': 0.8919648544524341, 'Filters': 0.5551629367883785, 'logP': 0.08376508833922258, 'SA': 0.04752492131394497, 'QED': 0.0039007607322941876, 'weight': 4.032424028268554, 'Novelty': 0.997643362136685, 'dataset': 'ames_50', 'generator_name': 'train1'}
bbb_martins_50
{'valid': 1.0, 'unique@1000': 0.9943661971830986, 'unique@10000': 0.9943661971830986, 'FCD/Test': 3.6841987379889076, 'SNN/Test': 0.5081955164146256, 'Frag/Test': 0.966294915338154, 'Scaf/Test': 0.5502519513284054, 'FCD/TestSF': 19.443867869643434, 'SNN/TestSF': 0.3730856233761764, 'Frag/TestSF': 0.7655736474564174, 'Scaf/TestSF': 0.0249430999943

In [21]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split ='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train2']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            print(dataset_name+split)
            graphs, splits =get_data(dataset_name+split, return_smiles=True)
            train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
            train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
            metrics=get_all_metrics(gen=train2_smiles,train=train2_smiles,test=train2_smiles)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.999, 'unique@10000': 0.9988221436984688, 'FCD/Test': 2.7000623958883807e-13, 'SNN/Test': 1.0, 'Frag/Test': 1, 'Scaf/Test': 1, 'FCD/TestSF': 22.184284328679503, 'SNN/TestSF': 0.33677946461981123, 'Frag/TestSF': 0.7366064124029432, 'Scaf/TestSF': 0.20333785426573803, 'IntDiv': 0.9102053951183938, 'IntDiv2': 0.8921645781772736, 'Filters': 0.5618374558303887, 'logP': 0.0, 'SA': 0.0, 'QED': 0.0, 'weight': 0.0, 'Novelty': 0.00039308176100628933, 'dataset': 'ames_50', 'generator_name': 'train2'}
bbb_martins_50
{'valid': 1.0, 'unique@1000': 0.9957805907172996, 'unique@10000': 0.9957805907172996, 'FCD/Test': 2.5579538487363607e-13, 'SNN/Test': 1.0, 'Frag/Test': 1, 'Scaf/Test': 1, 'FCD/TestSF': 20.862980852147757, 'SNN/TestSF': 0.3738190557404242, 'Frag/TestSF': 0.7047772230048593, 'Scaf/TestSF': 0.02450489523326016, 'IntDiv': 0.8879810418473504, 'IntDiv2': 0.8712773076208551, 'Filters': 0.7848101265822784, 'logP': 0.0, 'SA': 0.0, 'QED': 0.0, 'weight': 0.0

In [22]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split ='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train1/2+train2/2']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            print(dataset_name+split)
            graphs, splits =get_data(dataset_name+split, return_smiles=True)
            train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
            train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
            half=len(train1_smiles)//2
            train3_smiles=train1_smiles[:half] + train2_smiles[:half]
            metrics=get_all_metrics(gen=train3_smiles,train=train2_smiles,test=train2_smiles)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.999, 'unique@10000': 0.9972505891594659, 'FCD/Test': 1.4296308518017042, 'SNN/Test': 0.80414747127214, 'Frag/Test': 0.9769171525419632, 'Scaf/Test': 0.9058604214675904, 'FCD/TestSF': 24.652600888564585, 'SNN/TestSF': 0.3295319009622016, 'Frag/TestSF': 0.7325308897482011, 'Scaf/TestSF': 0.13555299909832097, 'IntDiv': 0.9005626242852879, 'IntDiv2': 0.8800491646635825, 'Filters': 0.4615082482325216, 'logP': 0.13850204932346513, 'SA': 0.06508225753225652, 'QED': 0.023545883072215484, 'weight': 11.541142909838637, 'Novelty': 0.4990153603781016, 'dataset': 'ames_50', 'generator_name': 'train1/2+train2/2'}
bbb_martins_50
{'valid': 1.0, 'unique@1000': 0.9929577464788732, 'unique@10000': 0.9929577464788732, 'FCD/Test': 3.7225768383014426, 'SNN/Test': 0.74866054233321, 'Frag/Test': 0.9741186915938969, 'Scaf/Test': 0.7575674931332002, 'FCD/TestSF': 19.48522466920629, 'SNN/TestSF': 0.378976951418838, 'Frag/TestSF': 0.6495216996091161, 'Scaf/TestSF': 0.007607

## Synthetic graphs 

In [13]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural', 'auc_roc']
generators=['swingnn']
splits=['_25','_33','_40','_50']
experiments=[5,10,25,33,40,50,66,75,100]
metrics={}
for generator in ['swingnn']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            for percent in experiments:
                a=[]
                print(dataset_name+split, percent)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_swingnn_graph_data(dataset_name,percent,split)
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100
                print(metrics)

synthetic_1_25 5
fid calculation produces singular product; adding 1e-06 to diagonal of cov estimates
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7061000000000001 0.7620000000000001 0.42635 0.5988499999999999
{'activations_time': 0.021608606974283854, 'fid': 1506.8639010864936, 'fid_time': 0.02366751035054525, 'kid': 53621.12933756511, 'kid_time': 0.029979753494262695, 'precision': 0.02805280528052806, 'recall': 0.09300000000000003, 'f1_pr': 0.039418427125137136, 'precision_time': 0.1307657241821289, 'recall_time': 0.1307657241821289, 'f1_pr_time': 0.1307657241821289, 'density': 0.010561056105610563, 'coverage': 0.012333333333333333, 'f1_dc': 0.011126430910821921, 'density_time': 0.0857300599416097, 'coverage_time': 0.0857300599416097, 'f1_dc_time': 0.0857300599416097, 'mmd_rbf': 1.1495767464240392, 'mmd_rbf_time': 0.11590353647867839, 'mmd_linear': 1310.2168090820312, 'mmd_linear_time': 0.021642700831095377, 'wl_mmd': 0.06129531767287366, 'wl_mmd_t

IndexError: list index out of range

In [34]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural', 'auc_roc']
generators=['swingnn']
splits=['_25']
experiments=[40,50,66,75,100]
metrics={}
for generator in ['swingnn']:
    for dataset_name in ['synthetic_2']:
        for split in splits:
            for percent in experiments:
                a=[]
                print(dataset_name+split, percent)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_swingnn_graph_data(dataset_name,percent,split)
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100
                print(metrics)

synthetic_2_25 40
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7066500000000001 0.8065000000000001 0.57715 0.70215
{'activations_time': 0.02081133524576823, 'fid': 53.54801262086392, 'fid_time': 0.021807336807250978, 'kid': 10.222130616505941, 'kid_time': 0.34615198771158856, 'precision': 0.9006600660066009, 'recall': 0.955, 'f1_pr': 0.9268044091823726, 'precision_time': 0.07230358123779297, 'recall_time': 0.07230358123779297, 'f1_pr_time': 0.07230358123779297, 'density': 0.7539933993399338, 'coverage': 0.9049999999999999, 'f1_dc': 0.821526563736224, 'density_time': 0.05179510116577148, 'coverage_time': 0.05179510116577148, 'f1_dc_time': 0.05179510116577148, 'mmd_rbf': 0.18772900104522705, 'mmd_rbf_time': 0.07443712552

In [35]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural', 'auc_roc']
generators=['swingnn']
splits=['_33','_40','_50']
experiments=[5,10,25,33,40,50,66,75,100]
metrics={}
for generator in ['swingnn']:
    for dataset_name in ['synthetic_2']:
        for split in splits:
            for percent in experiments:
                a=[]
                print(dataset_name+split, percent)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_swingnn_graph_data(dataset_name,percent,split)
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100
                print(metrics)

synthetic_2_33 5
fid calculation produces singular product; adding 1e-06 to diagonal of cov estimates
Now computing structural based metrics
Now computing the auc_roc based  metric
0.75395 0.7901 0.5714500000000001 0.7405999999999999
{'activations_time': 0.02593081792195638, 'fid': 120.28700067882339, 'fid_time': 0.02939001719156901, 'kid': 44.627427927652995, 'kid_time': 0.052056471506754555, 'precision': 0.6353383458646616, 'recall': 0.9797979797979798, 'f1_pr': 0.7704013847916878, 'precision_time': 0.11786653200785319, 'recall_time': 0.11786653200785319, 'f1_pr_time': 0.11786653200785319, 'density': 0.5438095238095239, 'coverage': 0.7257575757575757, 'f1_dc': 0.62114640300063, 'density_time': 0.08275324503580729, 'coverage_time': 0.08275324503580729, 'f1_dc_time': 0.08275324503580729, 'mmd_rbf': 0.34447813034057617, 'mmd_rbf_time': 0.12046810785929361, 'mmd_linear': 79.37751719156901, 'mmd_linear_time': 0.025968122482299804, 'wl_mmd': 0.009990954455827428, 'wl_mmd_time': 0.261246919

 ### Synthetic train2 vs train2

In [36]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural']
generators=['train2']
splits=['_25','_33','_40','_50']
experiments=[5,10,25,33,40,50,66,75,100]
metrics={}
for generator in ['train2']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            a=[]
            print(dataset_name+split)
            train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
            generated_graphs, generated_targets=train2_graphs,train2_targets
            splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
            for run in range(15):
                metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                a.append(metrics)
            metrics=dict_mean(a)
            metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)


synthetic_1_25
Now computing structural based metrics
{'activations_time': 0.022623825073242187, 'fid': -1.0144663657267e-06, 'fid_time': 0.025093936920166017, 'kid': -0.012700732549031575, 'kid_time': 0.03548707962036133, 'precision': 1.0, 'recall': 1.0, 'f1_pr': 1.0000099999999998, 'precision_time': 0.12578975359598796, 'recall_time': 0.12578975359598796, 'f1_pr_time': 0.12578975359598796, 'density': 1.2093999999999998, 'coverage': 1.0, 'f1_dc': 1.0946230169457278, 'density_time': 0.07988524436950684, 'coverage_time': 0.07988524436950684, 'f1_dc_time': 0.07988524436950684, 'mmd_rbf': 0.0, 'mmd_rbf_time': 0.11218183835347494, 'mmd_linear': 0.0, 'mmd_linear_time': 0.022661129633585613, 'wl_mmd': 0.0, 'wl_mmd_time': 0.2052440643310547, 'nspdk_mmd': 0.0, 'nspdk_mmd_time': 4.5912089347839355, 'degree_mmd': 0.0, 'degree_mmd_time': 7.127579, 'cluster_mmd': 0.0, 'cluster_mmd_time': 7.347449, 'spectral_mmd': 0.0, 'spectral_mmd_time': 7.874911, 'dataset': 'synthetic_1_25', 'generator_name': 't

# AMES_50_experiment

In [3]:
import torch
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from rdkit import RDLogger 
import os 
import pandas as pd 
import sys
import numpy as np
from rdkit import Chem
from rdkit.Chem.rdchem import ChiralType, BondType, BondDir
import networkx as nx
from copy import deepcopy

current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)
from sklearn.utils import shuffle

def string_targets_to_numeric(targets):
    targets=[eval(i) for i in targets]
    return targets

def get_data(name, folder, path='data/ames_50_exp/',return_smiles=False):
    splits={}
    RDLogger.EnableLog('rdApp.*')
    RDLogger.DisableLog('rdApp.*')  
    split_names=['train1_pos_smiles','train1_neg_smiles','train2_pos_smiles','train2_neg_smiles','test_pos_smiles','test_neg_smiles' ]
    for i,split in enumerate(split_names):
        exact_path=path+'{}/{}.txt'.format(name+folder, split)
        #from data.smiles.carcinogens import test_smiles
        current_list = []
        with open(exact_path) as my_file:
         for line in my_file:
            current_list.append(line.strip())
        splits[split]=current_list     
     
    #train_graphs=list_of_smiles_to_nx_graphs(splits['train_smiles'])
    #train_targets=string_targets_to_numeric(splits['train_tragets'])
    test_pos_graphs =list(list_of_smiles_to_nx_graphs(splits['test_pos_smiles']))
    test_neg_graphs =list(list_of_smiles_to_nx_graphs(splits['test_neg_smiles']))
    test_graphs = test_pos_graphs+ test_neg_graphs
    test_targets = np.array([1]*len(test_pos_graphs) + [0]*len(test_neg_graphs))
    test_graphs, test_targets = shuffle(test_graphs, test_targets,random_state=0)
    #valid_graphs=list_of_smiles_to_nx_graphs(splits['valid_smiles'])
    #valid_targets=string_targets_to_numeric(splits['valid_targets'])
    train1_pos_graphs =list(list_of_smiles_to_nx_graphs(splits['train1_pos_smiles']))
    train1_neg_graphs =list(list_of_smiles_to_nx_graphs(splits['train1_neg_smiles']))
    train1_graphs = train1_pos_graphs+ train1_neg_graphs
    train1_targets = np.array([1]*len(train1_pos_graphs) + [0]*len(train1_neg_graphs))
    train1_graphs, train1_targets = shuffle(train1_graphs, train1_targets,random_state=0)

    train2_pos_graphs =list(list_of_smiles_to_nx_graphs(splits['train2_pos_smiles']))
    train2_neg_graphs =list(list_of_smiles_to_nx_graphs(splits['train2_neg_smiles']))
    train2_graphs = train2_pos_graphs+ train2_neg_graphs
    train2_targets = np.array([1]*len(train2_pos_graphs) + [0]*len(train2_neg_graphs))
    train2_graphs, train2_targets = shuffle(train2_graphs, train2_targets,random_state=0)
   
    graphs=[train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets]
    if return_smiles:
         return graphs,splits
    else:  return graphs

In [4]:
def get_generated_data(name,folder, path='data/ames_50_exp/', generator_name='stgg',return_smiles=False):
    pos_list, neg_list=[],[]
    path_postives=path+'{}/{}_gen_pos_{}.txt'.format(name+folder,name+folder,generator_name)
    with open(path_postives) as my_file:
         for line in my_file:
            pos_list.append(line.strip())
    path_negatives=path+'{}/{}_gen_neg_{}.txt'.format(name+folder,name+folder,generator_name)
    with open(path_negatives) as my_file:
         for line in my_file:
            neg_list.append(line.strip())
    pos_graphs=list_of_smiles_to_nx_graphs(pos_list)
    neg_graphs=list_of_smiles_to_nx_graphs(neg_list)
    #print(len(neg_graphs))
    generated_graphs,generated_targets = pos_graphs[:250] + neg_graphs[:250], [1]*len(pos_graphs[:250] )+[0]*len(neg_graphs[:250] )
    #generated_graphs, generated_targets = shuffle(generated_graphs, generated_targets)
    if return_smiles:
         return  generated_graphs, generated_targets,pos_list+neg_list
    return  generated_graphs, np.array(generated_targets)

def dict_mean(dict_list):
    mean_dict = {}
    for key in dict_list[0].keys():
      try:
        mean_dict[key] = sum(float(d[key]) for d in dict_list) / len(dict_list)
      except: mean_dict[key]=dict_list[0][key]
    return mean_dict

## Nn and structural

In [5]:
current_smiles_datasets=['ames']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['swingnn']
folders=['_1',"_2","_3","_4","_5","_6","_7","_8"]
metrics={}
for generator in ['swingnn','stgg']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for folder in folders:
                
                    a=[]
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, folder, return_smiles=False)
                    generated_graphs, generated_targets=get_generated_data(dataset_name+split,folder,generator_name=generator)
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split+folder
                    metrics['generator_name']=generator
                    print(metrics)


ames_50 swingnn
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
{'activations_time': 0.03995860417683919, 'fid': 119.26725887800556, 'fid_time': 0.04256676038106282, 'kid': 6.586640858650208, 'kid_time': 0.34668628374735516, 'precision': 0.9786666666666666, 'recall': 0.8077333333333333, 'f1_pr': 0.8847987503510787, 'precision_time': 0.10849178632100423, 'recall_time': 0.10849178632100423, 'f1_pr_time': 0.10849178632100423, 'density': 1.038, 'coverage': 0.8140000000000001, 'f1_dc': 0.9121947492697545, 'density_time': 0.08598879178365072, 'coverage_time': 0.08598879178365072, 'f1_dc_time': 0.08598879178365072, 'mmd_rbf': 0.009606892173178495, 'mmd_rbf_time': 0.1149834156036377, 'mmd_linear': 0.4339665805300077, 'mmd_linear_tim

In [6]:
current_smiles_datasets=['ames']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['swingnn']
folders=['_1',"_2","_3","_4","_5","_6","_7","_8"]
metrics={}
for generator in ['train1']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for folder in folders:
                try:
                    a=[]
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, folder, return_smiles=False)
                    generated_graphs, generated_targets=train1_graphs, train1_targets
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split+folder
                    metrics['generator_name']=generator
                    print(metrics)
                except: None


ames_50 train1
Now computing classifier based metrics
Now computing structural based metrics
{'activations_time': 0.03599705696105957, 'fid': 4.08937552805888, 'fid_time': 0.03718873659769694, 'kid': -0.10012922286987305, 'kid_time': 0.05598154067993164, 'precision': 0.9762222222222221, 'recall': 0.9658666666666667, 'f1_pr': 0.9709915942585233, 'precision_time': 0.09466748237609864, 'recall_time': 0.09466748237609864, 'f1_pr_time': 0.09466748237609864, 'density': 1.0518666666666667, 'coverage': 0.9032, 'f1_dc': 0.9716862744964712, 'density_time': 0.07050889333089193, 'coverage_time': 0.07050889333089193, 'f1_dc_time': 0.07050889333089193, 'mmd_rbf': 0.005335874125739793, 'mmd_rbf_time': 0.10246562957763672, 'mmd_linear': 0.10587473561366399, 'mmd_linear_time': 0.03605966567993164, 'wl_mmd': 0.002781449582024309, 'wl_mmd_time': 0.21408891677856445, 'nspdk_mmd': 0.004713171457655541, 'nspdk_mmd_time': 2.457343101501465, 'degree_mmd': 0.00012135153822789846, 'degree_mmd_time': 7.883216, '

In [7]:
current_smiles_datasets=['ames']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
folders=['_1',"_2","_3","_4","_5","_6","_7","_8"]
metrics={}
for generator in ['train2']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for folder in folders:
                try:
                    a=[]
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, folder, return_smiles=False)
                    generated_graphs, generated_targets=train2_graphs, train2_targets
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split+folder
                    metrics['generator_name']=generator
                    print(metrics)
                except: None

ames_50 train2
Now computing classifier based metrics
Now computing structural based metrics
{'activations_time': 0.055671437581380205, 'fid': -1.4492239112466147e-09, 'fid_time': 0.061308431625366214, 'kid': -0.07753702799479166, 'kid_time': 0.08274688720703124, 'precision': 1.0, 'recall': 1.0, 'f1_pr': 1.0000099999999998, 'precision_time': 0.12667172749837238, 'recall_time': 0.12667172749837238, 'f1_pr_time': 0.12667172749837238, 'density': 1.1999999999999997, 'coverage': 1.0, 'f1_dc': 1.0909191735529684, 'density_time': 0.09203410148620605, 'coverage_time': 0.09203410148620605, 'f1_dc_time': 0.09203410148620605, 'mmd_rbf': 0.0, 'mmd_rbf_time': 0.13863186836242675, 'mmd_linear': 0.0, 'mmd_linear_time': 0.05572508176167806, 'wl_mmd': 0.0, 'wl_mmd_time': 0.47706007957458496, 'nspdk_mmd': 0.0, 'nspdk_mmd_time': 2.788238048553467, 'degree_mmd': 0.0, 'degree_mmd_time': 8.244806, 'cluster_mmd': 0.0, 'cluster_mmd_time': 8.705912, 'spectral_mmd': 0.0, 'spectral_mmd_time': 10.66038, 'dataset'

In [8]:
current_smiles_datasets=['ames']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
folders=['_1',"_2","_3","_4","_5","_6","_7","_8"]
metrics={}
for generator in ['train1/2+train2/2']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for folder in folders:
                try:
                    a=[]
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, folder, return_smiles=False)
                    half=len(train1_graphs)//2
                    half2=len(train2_graphs)//2
                    train3_graphs,train3_targets=train1_graphs[:half]+train2_graphs[:half], train1_targets[:half]+train2_targets[:half]
                    generated_graphs, generated_targets=train3_graphs, train3_targets
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split+folder
                    metrics['generator_name']=generator
                    print(metrics)
                except: None

ames_50 train1/2+train2/2
Now computing classifier based metrics
fid calculation produces singular product; adding 1e-06 to diagonal of cov estimates
Now computing structural based metrics
{'activations_time': 0.0364991823832194, 'fid': 1.6718290229733885, 'fid_time': 0.03853546778361003, 'kid': -0.10650673707326254, 'kid_time': 0.05433141390482585, 'precision': 0.9902222222222222, 'recall': 0.9811999999999999, 'f1_pr': 0.9856811086282378, 'precision_time': 0.09433107376098633, 'recall_time': 0.09433107376098633, 'f1_pr_time': 0.09433107376098633, 'density': 1.1271111111111112, 'coverage': 0.9596, 'f1_dc': 1.0364715417887165, 'density_time': 0.07121286392211915, 'coverage_time': 0.07121286392211915, 'f1_dc_time': 0.07121286392211915, 'mmd_rbf': 0.0033633310658236345, 'mmd_rbf_time': 0.10843933423360189, 'mmd_linear': 0.08007629836599033, 'mmd_linear_time': 0.03655085563659668, 'wl_mmd': 0.0011752062430435695, 'wl_mmd_time': 0.23468303680419922, 'nspdk_mmd': 0.002496365046338489, 'nspdk

## Molecular

In [9]:
from evaluation.moses.metrics.metrics import get_all_metrics

current_smiles_datasets=['ames']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
folders=['_1',"_2","_3","_4","_5","_6","_7","_8"]
metrics={}
for generator in ['swingnn','stgg']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for folder in folders:
                try:
                   
                    print(dataset_name+split, generator)
                    graphs, splits =get_data(dataset_name+split, folder, return_smiles=True)
                    train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
                    train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
                    _,_, generated_smiles=get_generated_data(dataset_name+split,folder,generator_name=generator, return_smiles=True)
                    metrics=get_all_metrics(gen=generated_smiles,train=train2_smiles,test=train2_smiles)
                    metrics['dataset']=dataset_name+split+folder
                    metrics['generator_name']=generator
                    print(metrics)
                except:continue


ames_50 swingnn
{'valid': 1.0, 'unique@1000': 0.444, 'unique@10000': 0.444, 'FCD/Test': nan, 'SNN/Test': 0.38134747920930384, 'Frag/Test': 0.8746937890254647, 'Scaf/Test': 0.30236944529712606, 'FCD/TestSF': nan, 'SNN/TestSF': 0.3369357900172472, 'Frag/TestSF': 0.6939810483631691, 'Scaf/TestSF': 0.04389825675094705, 'IntDiv': 0.9059081990852356, 'IntDiv2': 0.8653788410001039, 'Filters': 0.646, 'logP': 0.22781024000000002, 'SA': 0.10239513123588441, 'QED': 0.013569356107018277, 'weight': 7.006484000000005, 'Novelty': 1.0, 'dataset': 'ames_50_1', 'generator_name': 'swingnn'}
ames_50 swingnn
{'valid': 1.0, 'unique@1000': 0.48, 'unique@10000': 0.48, 'FCD/Test': nan, 'SNN/Test': 0.3865165919959545, 'Frag/Test': 0.8455301345148029, 'Scaf/Test': 0.39155380143285323, 'FCD/TestSF': nan, 'SNN/TestSF': 0.3247288521230221, 'Frag/TestSF': 0.7819419112624203, 'Scaf/TestSF': 0.11236639494356515, 'IntDiv': 0.9121401242599487, 'IntDiv2': 0.8720060435405692, 'Filters': 0.602, 'logP': 0.27077532000000004,

In [11]:
from evaluation.moses.metrics.metrics import get_all_metrics

current_smiles_datasets=['ames']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
folders=['_1',"_2","_3","_4","_5","_6","_7","_8"]
metrics={}
for generator in ['stgg']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for folder in folders:
                   
                    print(dataset_name+split, generator)
                    graphs, splits =get_data(dataset_name+split, folder, return_smiles=True)
                    train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
                    train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
                    _,_, generated_smiles=get_generated_data(dataset_name+split,folder,generator_name=generator, return_smiles=True)
                    metrics=get_all_metrics(gen=generated_smiles,train=train2_smiles,test=train2_smiles)
                    metrics['dataset']=dataset_name+split+folder
                    metrics['generator_name']=generator
                    print(metrics)


ames_50 stgg
{'valid': 1.0, 'unique@1000': 0.9153153153153153, 'unique@10000': 0.9153153153153153, 'FCD/Test': 6.704114492405914, 'SNN/Test': 0.33083531715579934, 'Frag/Test': 0.8702288550965989, 'Scaf/Test': 0.39006360351337177, 'FCD/TestSF': 27.00382401145815, 'SNN/TestSF': 0.2978177748419143, 'Frag/TestSF': 0.6795686839838438, 'Scaf/TestSF': 0.08835294471322408, 'IntDiv': 0.9083411542971251, 'IntDiv2': 0.8832883465010031, 'Filters': 0.4720720720720721, 'logP': 0.8278700603603608, 'SA': 0.6454627583186067, 'QED': 0.047546824260680375, 'weight': 54.608259333333336, 'Novelty': 0.9980314960629921, 'dataset': 'ames_50_1', 'generator_name': 'stgg'}
ames_50 stgg
{'valid': 1.0, 'unique@1000': 0.8788426763110307, 'unique@10000': 0.8788426763110307, 'FCD/Test': 6.181914402618531, 'SNN/Test': 0.33120959265592087, 'Frag/Test': 0.891149824979492, 'Scaf/Test': 0.3602313339009402, 'FCD/TestSF': 26.93512911258714, 'SNN/TestSF': 0.2965234329024663, 'Frag/TestSF': 0.6752885296262782, 'Scaf/TestSF': 0

In [10]:
from evaluation.moses.metrics.metrics import get_all_metrics

current_smiles_datasets=['ames']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
folders=['_1',"_2","_3","_4","_5","_6","_7","_8"]
metrics={}
for generator in ['train2']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for folder in folders:
                try:
                   
                    print(dataset_name+split, generator)
                    graphs, splits =get_data(dataset_name+split, folder, return_smiles=True)
                    train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
                    train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
                    metrics=get_all_metrics(gen=train2_smiles,train=train2_smiles,test=train2_smiles)
                    metrics['dataset']=dataset_name+split+folder
                    metrics['generator_name']=generator
                    print(metrics)
                except: None

ames_50 train2
{'valid': 1.0, 'unique@1000': 1.0, 'unique@10000': 1.0, 'FCD/Test': -3.730008302227361e-08, 'SNN/Test': 1.0, 'Frag/Test': 1, 'Scaf/Test': 1, 'FCD/TestSF': 25.169727439398457, 'SNN/TestSF': 0.3355734963417053, 'Frag/TestSF': 0.6862783989036914, 'Scaf/TestSF': 0.13278675590808064, 'IntDiv': 0.9110640195941925, 'IntDiv2': 0.8859189230564296, 'Filters': 0.556, 'logP': 0.0, 'SA': 0.0, 'QED': 0.0, 'weight': 0.0, 'Novelty': 0.004, 'dataset': 'ames_50_1', 'generator_name': 'train2'}
ames_50 train2
{'valid': 1.0, 'unique@1000': 1.0, 'unique@10000': 1.0, 'FCD/Test': -4.017100962983022e-08, 'SNN/Test': 1.0, 'Frag/Test': 1, 'Scaf/Test': 1, 'FCD/TestSF': nan, 'SNN/TestSF': 0.34780777409672736, 'Frag/TestSF': 0.6957713019326229, 'Scaf/TestSF': 0.0887653948693593, 'IntDiv': 0.9051048793334961, 'IntDiv2': 0.880854111644559, 'Filters': 0.618, 'logP': 0.0, 'SA': 0.0, 'QED': 0.0, 'weight': 0.0, 'Novelty': 0.0, 'dataset': 'ames_50_2', 'generator_name': 'train2'}
ames_50 train2
{'valid': 1.0

In [12]:
from evaluation.moses.metrics.metrics import get_all_metrics

current_smiles_datasets=['ames']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
folders=['_1',"_2","_3","_4","_5","_6","_7","_8"]
metrics={}
for generator in ['train1']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for folder in folders:
                try:
                   
                    print(dataset_name+split, generator)
                    graphs, splits =get_data(dataset_name+split, folder, return_smiles=True)
                    train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
                    train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
                    metrics=get_all_metrics(gen=train1_smiles,train=train2_smiles,test=train2_smiles)
                    metrics['dataset']=dataset_name+split+folder
                    metrics['generator_name']=generator
                    print(metrics)
                except: None

ames_50 train1
{'valid': 1.0, 'unique@1000': 1.0, 'unique@10000': 1.0, 'FCD/Test': nan, 'SNN/Test': 0.412440157532692, 'Frag/Test': 0.8955514521024635, 'Scaf/Test': 0.4804212260011963, 'FCD/TestSF': nan, 'SNN/TestSF': 0.34511964549620944, 'Frag/TestSF': 0.7460009362638043, 'Scaf/TestSF': 0.03202082488343416, 'IntDiv': 0.9055944295247396, 'IntDiv2': 0.87646597418375, 'Filters': 0.5966666666666667, 'logP': 0.13290337333333338, 'SA': 0.12059150853899157, 'QED': 0.012544576800898172, 'weight': 8.199979333333342, 'Novelty': 1.0, 'dataset': 'ames_50_1', 'generator_name': 'train1'}
ames_50 train1
{'valid': 1.0, 'unique@1000': 1.0, 'unique@10000': 1.0, 'FCD/Test': nan, 'SNN/Test': 0.41589521810412405, 'Frag/Test': 0.8999261405750852, 'Scaf/Test': 0.5190390395656187, 'FCD/TestSF': nan, 'SNN/TestSF': 0.3318534558514754, 'Frag/TestSF': 0.7463169344679127, 'Scaf/TestSF': 0.12547150287962372, 'IntDiv': 0.9116202979776594, 'IntDiv2': 0.88235269910618, 'Filters': 0.5366666666666666, 'logP': 0.1602537

In [13]:
from evaluation.moses.metrics.metrics import get_all_metrics

current_smiles_datasets=['ames']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
folders=['_1',"_2","_3","_4","_5","_6","_7","_8"]
metrics={}
for generator in ['train1/2+train2/2']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for folder in folders:
                try:
                   
                    print(dataset_name+split, generator)
                    graphs, splits =get_data(dataset_name+split, folder, return_smiles=True)
                    train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
                    train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
                    half=len(train1_smiles)//2
                    train3_smiles=train1_smiles[:half] + train2_smiles[:half]
                    metrics=get_all_metrics(gen=train3_smiles,train=train2_smiles,test=train2_smiles)
                    metrics['dataset']=dataset_name+split+folder
                    metrics['generator_name']=generator
                    print(metrics)
                except: None

ames_50 train1/2+train2/2
{'valid': 1.0, 'unique@1000': 1.0, 'unique@10000': 1.0, 'FCD/Test': nan, 'SNN/Test': 0.7154460494716962, 'Frag/Test': 0.8904943127893366, 'Scaf/Test': 0.7042864705322275, 'FCD/TestSF': nan, 'SNN/TestSF': 0.3298700954516729, 'Frag/TestSF': 0.7272768137080581, 'Scaf/TestSF': 0.050921883136147206, 'IntDiv': 0.9036916554027133, 'IntDiv2': 0.8719033620384706, 'Filters': 0.4666666666666667, 'logP': 0.17477593333333338, 'SA': 0.06052748039727778, 'QED': 0.023964689185863534, 'weight': 14.528852000000015, 'Novelty': 0.5, 'dataset': 'ames_50_1', 'generator_name': 'train1/2+train2/2'}
ames_50 train1/2+train2/2
{'valid': 1.0, 'unique@1000': 1.0, 'unique@10000': 1.0, 'FCD/Test': nan, 'SNN/Test': 0.7258219048380852, 'Frag/Test': 0.8851920329670939, 'Scaf/Test': 0.754152236850013, 'FCD/TestSF': nan, 'SNN/TestSF': 0.33130103195707006, 'Frag/TestSF': 0.7438859288569207, 'Scaf/TestSF': 0.06971362353298072, 'IntDiv': 0.8974925866232978, 'IntDiv2': 0.8669160861332977, 'Filters':

In [21]:
graphs, splits =get_data(dataset_name+split, folder, return_smiles=True)